# Task 2

In [1]:
from sklearn.externals import joblib

from scipy.stats import ks_2samp
import numpy as np
import datetime

from sklearn.neural_network import MLPClassifier

In [2]:
def getData(signal_filename, backgorund_filename):
    """
    :return: shuffled data
    """
    sig_data = np.asarray(joblib.load(signal_filename))
    bkg_data = np.asarray(joblib.load(backgorund_filename))
    np.random.shuffle(sig_data)
    np.random.shuffle(bkg_data)
    return sig_data, bkg_data

class Data:
    def __init__(self, trainFraction, feature1_idx, feature2_idx, sigLabel=-1, bkgLabel=1, signal_filename='', backgorund_filename=''):
        """
        :param trainFraction: float in (0,1)
        :param feature1_idx: int in [0,5]
        :param feature2_idx: int in [0,5]
        :param sigLabel
        :param bkgLabel
        """
        sig_data, bkg_data = getData(signal_filename, backgorund_filename)
        cutIndex = int(trainFraction * len(sig_data))
        self._sigTrain = sig_data[: cutIndex,:]
        np.random.shuffle(self._sigTrain)
        self._sigTest = sig_data[cutIndex:,:]
        self._bkgTrain = bkg_data[: cutIndex]
        np.random.shuffle(self._bkgTrain)
        self._bkgTest = bkg_data[cutIndex:,:]
        self._sigLabel = sigLabel
        self._bkgLabel = bkgLabel
        self._feature1_idx = feature1_idx
        self._feature2_idx = feature2_idx
        
    def set_feature_indexes(self, feature1_idx, feature2_idx):
        self._feature1_idx = feature1_idx
        self._feature2_idx = feature2_idx
        
    def shuffle(self):
        np.random.shuffle(self._sigTrain)
        np.random.shuffle(self._bkgTrain)
        
    def get_sigTrain(self):
        return self._sigTrain[:, (self._feature1_idx, self._feature2_idx)]
    
    def get_sigTest(self):
        return self._sigTest[:, (self._feature1_idx, self._feature2_idx)]
        
    def get_bkgTrain(self):
        return self._bkgTrain[:, (self._feature1_idx, self._feature2_idx)]
    
    def get_bkgTest(self):
        return self._bkgTest[:, (self._feature1_idx, self._feature2_idx)]
    
    def get_sigLabel(self):
        return self._sigLabel
    
    def get_bkgLabel(self):
        return self._bkgLabel

In [8]:
class Trainer:
        
    @staticmethod
    def train(hidden_layer_sizes, lr_init, dataObject, verbose):
        """
        Trains a classifier
        :param hidden_layer_sizes: tuple of zies: (100, 100)
        :param lr_init: initial learning rate: 0.3
        :return classifier
        """
        mlp = MLPClassifier(activation='relu', alpha=1e-05, batch_size=200,
                                   beta_1=0.9, beta_2=0.999, epsilon=1e-08,
                                   hidden_layer_sizes=hidden_layer_sizes, learning_rate_init=lr_init,
                                   random_state=1, shuffle=True, solver='adam', tol=0.00001,
                                   early_stopping=False, validation_fraction=0.1,
                                   verbose=verbose, warm_start=False)

        X = np.append(dataObject.get_sigTrain(), dataObject.get_bkgTrain(), axis=0)
        y = [dataObject.get_sigLabel()] * len(dataObject.get_sigTrain()) + [dataObject.get_bkgLabel()] * len(dataObject.get_bkgTrain())
        mlp.fit(X, y)
        return mlp

    @staticmethod
    def evaluate(classifier, dataObject, verbose):
        """
        :param classifier: MLPClassifier
        :return: test_accuracy
        """
        
        if len(dataObject.get_sigTrain()) != 0:
            predictions = []
            for entry in dataObject.get_sigTrain():
                predictions.append(classifier.predict([entry])[0])
            train_accuracy = predictions.count(dataObject.get_sigLabel()) / float(len(predictions))

        predictions = []
        for entry in dataObject.get_sigTest():
            predictions.append(classifier.predict([entry])[0])
        test_accuracy_sig = predictions.count(dataObject.get_sigLabel()) / float(len(predictions))

        if verbose:
            if len(dataObject.get_sigTrain()) != 0:
                print "Signal train accuracy: " + str(train_accuracy)
            print "Signal test accuracy: " + str(test_accuracy_sig)

        if len(dataObject.get_bkgTrain()) != 0:
            predictions = []
            for entry in dataObject.get_bkgTrain():
                predictions.append(classifier.predict([entry])[0])
            train_accuracy = predictions.count(dataObject.get_bkgLabel()) / float(len(predictions))

        predictions = []
        for entry in dataObject.get_bkgTest():
            predictions.append(classifier.predict([entry])[0])
        test_accuracy_bkg = predictions.count(dataObject.get_bkgLabel()) / float(len(predictions))

        if verbose:
            if len(dataObject.get_bkgTrain()) != 0:
                print "Background train accuracy: " + str(train_accuracy)
            print "Background test accuracy: " + str(test_accuracy_bkg)

        return (test_accuracy_bkg+test_accuracy_sig) / 2
    
    @staticmethod
    def predict_test_data(classifier, dataObject, verbose):
        """
        :param classifier: MLPClassifier
        :return: test_accuracy
        """
        
        testSample = []
        predictions_signal = []
        for entry in dataObject.get_sigTest():
            probability = float(classifier.predict_proba([entry])[0][0])
            predictions_signal.append(classifier.predict([entry])[0])
            testSample.append(probability)
        test_accuracy_sig = predictions_signal.count(dataObject.get_sigLabel()) / float(len(predictions_signal))

        if verbose:
            print "Signal test accuracy: " + str(test_accuracy_sig)

        testSample = []
        predictions_background = []
        for entry in dataObject.get_bkgTest():
            probability = float(classifier.predict_proba([entry])[0][0])
            predictions_background.append(classifier.predict([entry])[0])
            testSample.append(probability)
        test_accuracy_bkg = predictions_background.count(dataObject.get_bkgLabel()) / float(len(predictions_background))

        if verbose:
            print "Background test accuracy: " + str(test_accuracy_bkg)

        return (test_accuracy_bkg+test_accuracy_sig) / 2, predictions_signal, dataObject._sigTest, predictions_background, dataObject._bkgTest
    
    @staticmethod
    def saveClassifier(classifier, filename):
        joblib.dump(classifier, filename )
        print 'Classifier saved to file'
    
    @staticmethod
    def loadClassifier(filename):
        classifier = joblib.load(filename )
        return classifier

In [4]:
def train_N(N, layer_sizes, lr_init, dataObject, verbose):
    accuracy_setting_history = []
    for _ in range(N):
        classifier = Trainer.train(layer_sizes,lr_init, dataObject, verbose)
        accuracy = Trainer.evaluate(classifier, dataObject, verbose)
        accuracy_setting_history.append(accuracy)

    candidate = sum(accuracy_setting_history)/N
    return classifier, candidate

In [5]:
def hyperparameter_search():
    trainFraction_ = 0.5
    hidden_layer_sizes_ = (100, 100)
    lr_init_ = 0.3

    histories_indexes_ = []
    best_average_accuracy_ = 0
    best_feature1_idx_ = -1
    best_feature2_idx_ = -1

    dataObject_ = Data(trainFraction_, best_feature1_idx_, best_feature2_idx_)
    
    for feature1_idx_ in range(6):
        for feature2_idx_ in range(feature1_idx_+1, 6):

            dataObject_.set_feature_indexes(feature1_idx_, feature2_idx_)
            _, candidate_ = train_N(1, hidden_layer_sizes_, lr_init_, dataObject_, verbose=False)
            dataObject_.shuffle()
            if candidate_ > best_average_accuracy_:
                best_average_accuracy_ = candidate_
                best_feature1_idx_ = feature1_idx_
                best_feature2_idx_ = feature2_idx_

            histories_indexes_.append([feature1_idx_, feature2_idx_, candidate_])
    
    #print "(feature1, feature2, AP)"
    #print histories_indexes_
    print "Best feature indexes "+ str(best_feature1_idx_) + " " + str(best_feature2_idx_)
    #print "Best accuracy " + str(best_average_accuracy_)
    
    network_dims_ = [(200, 200), (10,10), (20,20,20,20), (100, 40, 20, 10), ]

    histories_hidden_sizes_ = []
    best_sizes_ = None
    best_average_accuracy_ = 0

    dataObject_.set_feature_indexes(best_feature1_idx_, best_feature2_idx_)

    for hidden_sizes_ in network_dims_:

        _, candidate_ = train_N(1, hidden_sizes_, lr_init_, dataObject_, verbose=False)
        dataObject_.shuffle()
        if candidate_ > best_average_accuracy_:
            best_average_accuracy_ = candidate_
            best_sizes_ = hidden_sizes_

        histories_hidden_sizes_.append([hidden_sizes_, candidate_])
    
    #print "((hidden layer sizes), AP)"
    #print histories_hidden_sizes_
    print "Best hidden layer size " + str(best_sizes_)
    
    return best_feature1_idx_, best_feature2_idx_, best_sizes_

# 1. Training a Input Model ( Neural Network that is taught on the signal )
## Skip following 3 cells if you don't wanna train and search hyperparameters

In [117]:
best_feature1_idx_, best_feature2_idx_, best_sizes_ = hyperparameter_search()

Best feature indexes 2 3
Best hidden layer size (200, 200)


In [118]:
print (best_feature1_idx_, best_feature2_idx_, best_sizes_)

(2, 3, (200, 200))


In [119]:
def save_best_model(best_feature1_idx_, best_feature2_idx_, best_sizes_):
    dataObject_ = Data(trainFraction_, best_feature1_idx_, best_feature2_idx_)
    dataObject_.set_feature_indexes(best_feature1_idx_, best_feature2_idx_)
    best_classifier_ = Trainer.train(best_sizes_, lr_init_, dataObject_, verbose=False)
    best_accuracy_ = Trainer.evaluate(best_classifier_, dataObject_, verbose=False)
    print "Best model accuracy: " + str(best_accuracy_)
    Trainer.saveClassifier(best_classifier_,'best_classifier2.pkl')
save_best_model(best_feature1_idx_, best_feature2_idx_, best_sizes_)

Best model accuracy: 0.8962
Classifier saved to file


### Run from here if you want to evaluate best model on new data with the same format 
### the filenames with filepaths should be sent as parameters: signal file path, background file path

In [9]:
def evaluate_best_classifier_on_new_data(signal_filename, backgorund_filename, classifier_filename):
    dataObject_ = Data(trainFraction=0, feature1_idx=2, feature2_idx=3, signal_filename=signal_filename, backgorund_filename=backgorund_filename)
    loaded_class_ = Trainer.loadClassifier(classifier_filename)
    accuracy_ = Trainer.evaluate(loaded_class_, dataObject_, verbose=True)
    print "Accuracy: "+str(accuracy_)

In [10]:
evaluate_best_classifier_on_new_data('./HEPDrone/data/signal_data.p', './HEPDrone/data/background_data.p', 'best_classifier2.pkl')

Signal test accuracy: 0.9038
Background test accuracy: 0.8879
Accuracy: 0.89585
